# 2. Preprocesamiento de Datos

**Objetivo:**  
Construir un pipeline de preprocesamiento usando `sklearn` que transforme 
las variables del dataset para que sean aptas para el modelado de Machine Learning.

**Tareas realizadas:**
1. Separación de variables predictoras (X) y variable objetivo (y)
2. División de datos en conjunto de entrenamiento (80%) y test (20%)
3. Construcción de Pipeline con ColumnTransformer:
   - `OneHotEncoder` para variables categóricas
   - `StandardScaler` para variables numéricas
4. Serialización del preprocesador entrenado

**Clasificación de variables:**

| Variable | Tipo | Transformación |
|----------|------|----------------|
| PetType | Categórica | OneHotEncoder |
| Breed | Categórica | OneHotEncoder |
| Color | Categórica | OneHotEncoder |
| Size | Categórica | OneHotEncoder |
| AgeMonths | Numérica | StandardScaler |
| WeightKg | Numérica | StandardScaler |
| AdoptionFee | Numérica | StandardScaler |
| Vaccinated | Binaria | Sin transformación |
| HealthCondition | Binaria | Sin transformación |
| PreviousOwner | Binaria | Sin transformación |
| PetID | Identificador | Eliminada |
| AdoptionLikelihood | Otra variable objetivo | Eliminada |
| TimeInShelterDays | Variable objetivo (y) | Sin transformación |

---

In [12]:
# ============================================
# Importaciones y Carga de Datos
# ============================================

import numpy as np
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


#Cargamos nuesro dataset
df = pd.read_csv('../data/pet_adoption_data.csv')

print(f"Dataset cargado: {df.shape[0]} filas x {df.shape[1]} columnas")


Dataset cargado: 2007 filas x 13 columnas


## 2.1 Separación de Variables: X e y

**Objetivo:**  
Separar el dataset en variables predictoras (X) y variable objetivo (y).

**Decisiones tomadas:**
- `TimeInShelterDays` → Variable objetivo (y)
- `PetID` → Eliminada (identificador sin valor predictivo)
- `AdoptionLikelihood` → Eliminada (variable objetivo alternativa, no predictora)
- Resto de columnas → Variables predictoras (X)

---

## 2.1 Separación de Variables: X e y

**Objetivo:**  
Separar el dataset en variables predictoras (X) y variable objetivo (y).

**Decisiones tomadas:**
- `TimeInShelterDays` → Variable objetivo (y)
- `PetID` → Eliminada (identificador sin valor predictivo)
- `AdoptionLikelihood` → Eliminada (variable objetivo alternativa, no predictora)
- Resto de columnas → Variables predictoras (X)


In [13]:
# ============================================
# Separación de Variables X e y
# ============================================

# Variable objetivo
y = df['TimeInShelterDays']

# Variables predictoras
# Eliminamos: PetID (identificador), 
#             TimeInShelterDays (es la y),
#             AdoptionLikelihood (otra variable objetivo)
X = df.drop(columns=['PetID', 'TimeInShelterDays', 'AdoptionLikelihood'])

# Verificar resultado
print("="*60)
print("SEPARACIÓN X e y")
print("="*60)

print(f"\n📊 VARIABLE OBJETIVO (y):")
print(f"   Nombre:    TimeInShelterDays")
print(f"   Registros: {y.shape[0]}")
print(f"   Rango:     {y.min()} - {y.max()} días")

print(f"\n📊 VARIABLES PREDICTORAS (X):")
print(f"   Dimensiones: {X.shape[0]} filas x {X.shape[1]} columnas")
print(f"   Columnas: {list(X.columns)}")

SEPARACIÓN X e y

📊 VARIABLE OBJETIVO (y):
   Nombre:    TimeInShelterDays
   Registros: 2007
   Rango:     1 - 89 días

📊 VARIABLES PREDICTORAS (X):
   Dimensiones: 2007 filas x 10 columnas
   Columnas: ['PetType', 'Breed', 'AgeMonths', 'Color', 'Size', 'WeightKg', 'Vaccinated', 'HealthCondition', 'AdoptionFee', 'PreviousOwner']


### Interpretación

**Variable objetivo (y):**
- `TimeInShelterDays`: variable continua con rango [1, 89] días
- Es lo que el modelo aprenderá a predecir

**Variables predictoras (X):**
- Total: 10 columnas
- 4 categóricas: PetType, Breed, Color, Size
- 3 numéricas: AgeMonths, WeightKg, AdoptionFee
- 3 binarias: Vaccinated, HealthCondition, PreviousOwner

**Variables eliminadas:**
- `PetID`: identificador único sin valor predictivo
- `AdoptionLikelihood`: variable objetivo alternativa que 
  no debe usarse como predictor para evitar data leakage

---

## 2.2 División Train/Test (80/20)

**Objetivo:**  
Dividir el dataset en dos conjuntos independientes para entrenamiento y evaluación del modelo.

**Configuración:**
- **80% (1605 registros)** → Conjunto de entrenamiento (TRAIN)
  - El modelo aprende con estos datos
- **20% (402 registros)** → Conjunto de prueba (TEST)
  - El modelo se evalúa con datos que nunca ha visto
- **random_state=42** → Semilla aleatoria para reproducibilidad
  - Garantiza que la división sea siempre la misma

**¿Por qué dividir?**

Sin división (usar todos los datos para entrenar):
- El modelo memoriza los datos (OVERFITTING)
- No sabemos si funcionará con datos nuevos

Con división:
- Evaluamos el modelo con datos que no ha visto
- Detectamos si realmente aprendió o solo memorizó

---

In [14]:
# ============================================
# División Train/Test (80/20)
# ============================================

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,       #20% para test
    random_state=42      #Semilla para reproducibilidad
)

#Verificar resultado
print("="*60)
print("DIVISIÓN TRAIN/TEST")
print("="*60)

print(f"\n📊 CONJUNTO DE ENTRENAMIENTO:")
print(f"   X_train: {X_train.shape[0]} filas x {X_train.shape[1]} columnas")
print(f"   y_train: {y_train.shape[0]} valores")
print(f"   Porcentaje: {X_train.shape[0]/len(X)*100:.1f}%")

print(f"\n📊 CONJUNTO DE PRUEBA:")
print(f"   X_test: {X_test.shape[0]} filas x {X_test.shape[1]} columnas")
print(f"   y_test: {y_test.shape[0]} valores")
print(f"   Porcentaje: {X_test.shape[0]/len(X)*100:.1f}%")

print(f"\n📊 ESTADÍSTICAS DE LA VARIABLE OBJETIVO:")
print(f"   y_train - Media: {y_train.mean():.2f} días, Std: {y_train.std():.2f}")
print(f"   y_test  - Media: {y_test.mean():.2f} días, Std: {y_test.std():.2f}")


DIVISIÓN TRAIN/TEST

📊 CONJUNTO DE ENTRENAMIENTO:
   X_train: 1605 filas x 10 columnas
   y_train: 1605 valores
   Porcentaje: 80.0%

📊 CONJUNTO DE PRUEBA:
   X_test: 402 filas x 10 columnas
   y_test: 402 valores
   Porcentaje: 20.0%

📊 ESTADÍSTICAS DE LA VARIABLE OBJETIVO:
   y_train - Media: 44.23 días, Std: 25.75
   y_test  - Media: 42.96 días, Std: 25.72


### Interpretación

**División realizada correctamente:**
- Train: 1605 registros (80%)
- Test: 402 registros (20%)

**Validación de la división:**

La media y desviación estándar de `y_train` y `y_test` deben ser similares, 
lo que indica que la división fue aleatoria y representativa del dataset completo.

Si las estadísticas fueran muy diferentes, indicaría un sesgo en la división 
(ejemplo: todos los valores altos en train y los bajos en test).

**random_state=42:**

Al fijar la semilla aleatoria, garantizamos que:
- Cualquier persona que ejecute este notebook obtendrá la misma división
- Los resultados son reproducibles
- Es esencial para la validación científica del modelo

---

## 2.3 Construcción del Pipeline de Preprocesamiento

**Objetivo:**  
Crear un `ColumnTransformer` que aplique transformaciones diferentes a diferentes tipos de columnas automáticamente.

**Estrategia de transformación:**

| Tipo de Variable | Columnas | Transformación | Justificación |
|------------------|----------|----------------|---------------|
| **Categóricas** | PetType, Breed, Color, Size | `OneHotEncoder` | Convierte texto en columnas binarias (0/1) |
| **Numéricas** | AgeMonths, WeightKg, AdoptionFee | `StandardScaler` | Estandariza a media=0, std=1 |
| **Binarias** | Vaccinated, HealthCondition, PreviousOwner | `passthrough` | Ya están en escala 0/1, no requieren transformación |

**Pipeline de sklearn:**

Un Pipeline encadena transformaciones de forma automática y ordenada:
1. Identifica qué columnas son de cada tipo
2. Aplica la transformación correspondiente a cada grupo
3. Combina todo en una matriz lista para el modelo

**Ventajas:**
- Evita errores manuales
- Garantiza que train y test reciban las mismas transformaciones
- Facilita la serialización para producción

---

In [15]:
# ============================================
# Definir Columnas por Tipo
# ============================================

# Categóricas (texto) → OneHotEncoder
categorical_features = ['PetType', 'Breed', 'Color', 'Size']

# Numéricas (escalar) → StandardScaler
numerical_features = ['AgeMonths', 'WeightKg', 'AdoptionFee']

# Binarias (0/1) → Sin transformación
binary_features = ['Vaccinated', 'HealthCondition', 'PreviousOwner']

print("="*60)
print("DEFINICIÓN DE COLUMNAS POR TIPO")
print("="*60)
print(f"\nCategóricas ({len(categorical_features)}): {categorical_features}")
print(f"Numéricas ({len(numerical_features)}): {numerical_features}")
print(f"Binarias ({len(binary_features)}): {binary_features}")

# ============================================
# Construir ColumnTransformer
# ============================================

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), 
         categorical_features),
        ('num', StandardScaler(), 
         numerical_features),
        ('bin', 'passthrough', 
         binary_features)
    ],
    remainder='drop'  # Eliminar cualquier columna no especificada
)

print("\n" + "="*60)
print("PIPELINE CONSTRUIDO")
print("="*60)
print(f"\n✅ ColumnTransformer creado con 3 transformadores:")
print(f"   1. OneHotEncoder   → {len(categorical_features)} columnas categóricas")
print(f"   2. StandardScaler  → {len(numerical_features)} columnas numéricas")
print(f"   3. Passthrough     → {len(binary_features)} columnas binarias")

DEFINICIÓN DE COLUMNAS POR TIPO

Categóricas (4): ['PetType', 'Breed', 'Color', 'Size']
Numéricas (3): ['AgeMonths', 'WeightKg', 'AdoptionFee']
Binarias (3): ['Vaccinated', 'HealthCondition', 'PreviousOwner']

PIPELINE CONSTRUIDO

✅ ColumnTransformer creado con 3 transformadores:
   1. OneHotEncoder   → 4 columnas categóricas
   2. StandardScaler  → 3 columnas numéricas
   3. Passthrough     → 3 columnas binarias


In [16]:
# ============================================
# Entrenar el Preprocesador (FIT)
# ============================================

preprocessor.fit(X_train)

print("="*60)
print("PREPROCESADOR ENTRENADO")
print("="*60)
print("\n✅ Fit completado con X_train")
print(f"   Registros utilizados: {X_train.shape[0]}")

# Ver cúantas columnas se generaron después de OneHotEncoder
X_train_transformed = preprocessor.transform(X_train)
print(f"\n📊 RESULTADO DE LA TRANSFORMACIÓN:")
print(f"   Columnas ANTES:   {X_train.shape[1]}")
print(f"   Columnas DESPUÉS: {X_train_transformed.shape[1]}")
print(f"\n   ¿Por qué más columnas?")
print(f"   → OneHotEncoder convirtió las 4 categóricas")
print(f"     en múltiples columnas binarias (una por categoría)")


PREPROCESADOR ENTRENADO

✅ Fit completado con X_train
   Registros utilizados: 1605

📊 RESULTADO DE LA TRANSFORMACIÓN:
   Columnas ANTES:   10
   Columnas DESPUÉS: 21

   ¿Por qué más columnas?
   → OneHotEncoder convirtió las 4 categóricas
     en múltiples columnas binarias (una por categoría)


In [17]:
X_train.head()

,PetType,Breed,AgeMonths,Color,Size,WeightKg,Vaccinated,HealthCondition,AdoptionFee,PreviousOwner
916,Dog,Golden Retriever,68,Black,Medium,15.083291,1,0,419,0
261,Cat,Siamese,172,Brown,Medium,24.471717,0,0,408,1
607,Rabbit,Rabbit,169,Orange,Large,10.893330,0,0,373,0
1331,Rabbit,Rabbit,135,Gray,Large,2.863329,0,1,342,0
240,Rabbit,Rabbit,169,Orange,Small,26.926209,1,0,55,0


### Interpretación del Pipeline

**¿Qué hace `drop='first'` en OneHotEncoder?**

Evita redundancia eliminando una categoría de referencia.

Ejemplo con PetType:
```
Sin drop='first':
PetType_Dog  PetType_Cat  PetType_Rabbit
     1            0            0         ← Dog
     0            1            0         ← Cat
     0            0            1         ← Rabbit

Con drop='first' (elimina Dog):
PetType_Cat  PetType_Rabbit
     0            0         ← Dog (se infiere: no es Cat ni Rabbit)
     1            0         ← Cat
     0            1         ← Rabbit
```

Esto evita multicolinealidad perfecta en modelos de regresión.

**¿Qué hace `handle_unknown='ignore'`?**

Si en producción llega una categoría nueva que no existía en train 
(ejemplo: una raza nueva), en lugar de dar error, la ignora y 
asigna 0 a todas las columnas.

**¿Por qué `fit` solo con X_train?**
```
❌ MAL: preprocessor.fit(X)  
   → El preprocesador "ve" los datos de test
   → Data leakage: información del futuro filtra al entrenamiento

✅ BIEN: preprocessor.fit(X_train)
   → El preprocesador solo "ve" los datos de train
   → Simula condiciones reales donde el test es futuro desconocido
```

**Aumento de columnas:**

Las 10 columnas originales se expandieron a **21 columnas** 
debido a que OneHotEncoder convirtió cada valor único de las categóricas 
en una columna binaria separada.

---

## 2.4 Serialización del Preprocesador

**Objetivo:**  
Guardar el preprocesador entrenado en un archivo para poder reutilizarlo sin necesidad de reentrenarlo.

**¿Qué es serializar?**

Serializar = Convertir un objeto de Python (el preprocesador) en un archivo binario que se puede:
- Guardar en disco
- Cargar posteriormente
- Usar en producción (app Streamlit)

**¿Por qué serializar?**
```
Sin serialización:
→ Cada vez que abres el notebook hay que ejecutar todo
→ En producción tendrías que reentrenar cada vez
→ Pérdida de tiempo y recursos

Con serialización:
→ Guardas el preprocesador una vez
→ Lo cargas instantáneamente cuando lo necesitas
→ La app Streamlit lo usa directamente
```

**Librería utilizada:**

`joblib` es la librería recomendada por sklearn para serializar modelos y preprocesadores.
Es más eficiente que `pickle` para objetos grandes con arrays de NumPy.

---

In [18]:
# ============================================
# Serializar el Preprocesador
# ============================================

# Crear carpeta models/ si no existe
os.makedirs('../models', exist_ok=True)

# Guardar el preprocesador
preprocessor_path = '../models/preprocessor.joblib'
joblib.dump(preprocessor, preprocessor_path)

print("="*60)
print("SERIALIZACIÓN COMPLETADA")
print("="*60)
print(f"\n✅ Preprocesador guardado en:")
print(f"   {preprocessor_path}")

# Verificar que se guardó correctamente
file_size = os.path.getsize(preprocessor_path) / 1024  # Tamaño en KB
print(f"\n📊 INFORMACIÓN DEL ARCHIVO:")
print(f"   Tamaño: {file_size:.2f} KB")
print(f"   Ruta absoluta: {os.path.abspath(preprocessor_path)}")

# ============================================
# Prueba de Carga (verificación)
# ============================================

# Cargar el preprocesador para verificar que funciona
preprocessor_loaded = joblib.load(preprocessor_path)

# Probar transformación con los primeros 5 registros de test
X_test_sample = X_test.head(5)
X_test_transformed = preprocessor_loaded.transform(X_test_sample)

print(f"\n✅ VERIFICACIÓN DE CARGA:")
print(f"   Preprocesador cargado correctamente")
print(f"   Prueba de transformación: 5 registros → {X_test_transformed.shape[1]} columnas")
print(f"\n   El preprocesador está listo para usar en producción.")

SERIALIZACIÓN COMPLETADA

✅ Preprocesador guardado en:
   ../models/preprocessor.joblib

📊 INFORMACIÓN DEL ARCHIVO:
   Tamaño: 4.31 KB
   Ruta absoluta: c:\Users\SONY VAIO\desktop\ia\practicas\Proyect_V_Regression_Team3\models\preprocessor.joblib

✅ VERIFICACIÓN DE CARGA:
   Preprocesador cargado correctamente
   Prueba de transformación: 5 registros → 21 columnas

   El preprocesador está listo para usar en producción.


### Resumen del Preprocesamiento

**Pipeline completado exitosamente:**

1. ✅ **Separación X e y**
   - Variable objetivo: `TimeInShelterDays` (2007 registros)
   - Variables predictoras: 10 columnas

2. ✅ **División Train/Test**
   - Train: 1605 registros (80%)
   - Test: 402 registros (20%)
   - División balanceada (medias similares)

3. ✅ **ColumnTransformer construido**
   - OneHotEncoder: 4 categóricas → expandidas
   - StandardScaler: 3 numéricas → estandarizadas
   - Passthrough: 3 binarias → sin cambios
   - Resultado: 10 columnas → 21 columnas

4. ✅ **Preprocesador serializado**
   - Archivo: `models/preprocessor.joblib`
   - Listo para usar en notebooks posteriores
   - Listo para producción (Streamlit)

